# TD0 — Introduction : De l'OLTP au Décisionnel (DWH/OLAP)

**Durée estimée** : 1h30  
**Niveau** : Débutant  
**Objectifs pédagogiques** :
1. Comprendre le fonctionnement d'une base transactionnelle (OLTP) et ses limites pour l'analyse.
2. Mesurer l'impact des jointures et de la normalisation sur les performances analytiques.
3. Découvrir l'intérêt d'un modèle dénormalisé (Entrepôt de Données / Star Schema).
4. Établir un plan de transition vers la Business Intelligence.

---

## Partie 1 : Le monde Transactionnel (OLTP)

Les systèmes **OLTP** (Online Transaction Processing) sont conçus pour gérer les opérations quotidiennes de l'entreprise (commandes, paiements, stocks).

### Caractéristiques clés :
- **Atomicité** : Des transactions courtes et sécurisées (ACID).
- **Normalisation** : Les données sont fragmentées en plusieurs tables (3NF) pour éviter la redondance et garantir la cohérence.
- **Mise à jour fréquente** : Beaucoup d'écritures (INSERT, UPDATE).

### Modèle de données (Exemple E-Commerce)
Voici à quoi ressemble un schéma classique pour gérer des ventes. Remarquez la séparation entre `commandes`, `lignes_commande`, `produits` et `clients`.

```mermaid
erDiagram
    CLIENT {
        int client_id PK
        string nom
        string ville
        string segment
    }
    PRODUIT {
        int produit_id PK
        string nom
        string categorie
        float prix_unitaire
    }
    COMMANDE {
        int commande_id PK
        int client_id FK
        date date_commande
        string statut
    }
    LIGNE_COMMANDE {
        int commande_id FK
        int produit_id FK
        int quantite
        float prix_reel
    }

    CLIENT ||--o{ COMMANDE : passe
    COMMANDE ||--|{ LIGNE_COMMANDE : contient
    PRODUIT ||--o{ LIGNE_COMMANDE : est_reference_dans
```

## Partie 2 : Le problème analytique

Si l'on veut répondre à une question simple comme **"Quel est le Chiffre d'Affaires mensuel par catégorie de produit et par ville ?"**, le modèle OLTP devient un obstacle.

**Pourquoi ?**
1. Il faut **reconstituer l'information** éparpillée (Jointures).
2. Il faut **scanner beaucoup de lignes** pour agréger (Group By).
3. Les index sont souvent optimisés pour l'écriture (ID), pas pour l'analyse (Date, Catégorie).

### Démonstration pratique (Python/SQLite)
Nous allons simuler une petite base de données en mémoire pour visualiser le problème.

In [ ]:
import sqlite3
import pandas as pd

# Connexion à une base en mémoire (rapide pour la démo)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

print("🚀 Création du schéma OLTP...")

# 1. Création des tables (Normalisées)
cursor.executescript('''
-- Table Clients
CREATE TABLE clients (
    client_id INTEGER PRIMARY KEY,
    nom TEXT,
    ville TEXT,
    segment TEXT
);

-- Table Produits
CREATE TABLE produits (
    produit_id INTEGER PRIMARY KEY,
    nom TEXT,
    categorie TEXT,
    prix_standard REAL
);

-- Table Commandes (Entête)
CREATE TABLE commandes (
    commande_id INTEGER PRIMARY KEY,
    client_id INTEGER,
    date_commande DATE,
    statut TEXT,
    FOREIGN KEY(client_id) REFERENCES clients(client_id)
);

-- Table Lignes de Commande (Détail)
CREATE TABLE lignes_commande (
    commande_id INTEGER,
    produit_id INTEGER,
    quantite INTEGER,
    prix_reel REAL,
    FOREIGN KEY(commande_id) REFERENCES commandes(commande_id),
    FOREIGN KEY(produit_id) REFERENCES produits(produit_id)
);
''')

# 2. Insertion de données (Jeu d'essai)
cursor.executescript('''
INSERT INTO clients VALUES 
    (1, 'Alice Dupont', 'Paris', 'VIP'),
    (2, 'Bob Martin', 'Lyon', 'Standard'),
    (3, 'Charlie Durand', 'Paris', 'Standard');

INSERT INTO produits VALUES 
    (10, 'Laptop Pro', 'Électronique', 1200.0),
    (11, 'Smartphone X', 'Électronique', 800.0),
    (12, 'Chaise Bureau', 'Mobilier', 150.0);

INSERT INTO commandes VALUES 
    (1001, 1, '2024-01-15', 'LIVRE'),
    (1002, 2, '2024-01-16', 'LIVRE'),
    (1003, 1, '2024-02-01', 'EN_COURS');

INSERT INTO lignes_commande VALUES 
    (1001, 10, 1, 1200.0), -- Alice achète 1 Laptop
    (1001, 12, 2, 140.0),  -- Alice achète 2 Chaises (promo)
    (1002, 11, 1, 800.0),  -- Bob achète 1 Smartphone
    (1003, 12, 1, 150.0);  -- Alice achète 1 Chaise
''')

print("✅ Données OLTP chargées avec succès.")

In [ ]:
# Requête Analytique sur OLTP : CA par Mois, Catégorie et Ville
# Remarquez le nombre de JOINTURES nécessaires !

sql_oltp = '''
SELECT 
    strftime('%Y-%m', c.date_commande) as mois,
    p.categorie,
    cl.ville,
    SUM(lc.quantite * lc.prix_reel) as chiffre_affaires,
    COUNT(DISTINCT c.commande_id) as nb_commandes
FROM commandes c
JOIN lignes_commande lc ON c.commande_id = lc.commande_id
JOIN produits p ON lc.produit_id = p.produit_id
JOIN clients cl ON c.client_id = cl.client_id
GROUP BY 
    strftime('%Y-%m', c.date_commande),
    p.categorie,
    cl.ville
ORDER BY mois, chiffre_affaires DESC;
'''

print("📊 Exécution de la requête OLTP (Complexe)...")
df_oltp = pd.read_sql_query(sql_oltp, conn)
display(df_oltp)

## Partie 3 : La Solution Décisionnelle (Pré-OLAP)

Pour résoudre ces problèmes de performance et de complexité, nous allons créer une structure dédiée à l'analyse : **l'Entrepôt de Données (Data Warehouse)**.

L'approche consiste à :
1. **Dénormaliser** : Regrouper les informations liées dans une même structure.
2. **Pré-calculer** : Si possible, préparer les données (ETL).
3. **Simplifier** : Utiliser un modèle en étoile (Faits + Dimensions).

Dans cet exemple simplifié, nous allons créer une table unique `fact_ventes` qui matérialise toutes les informations nécessaires.

# Simulation d'un ETL (Extract-Transform-Load) vers une table de faits
# On "aplatit" le modèle pour l'analyse

cursor.execute('''
CREATE TABLE fact_ventes AS
SELECT 
    lc.produit_id,
    lc.commande_id,
    c.client_id,
    c.date_commande,
    p.categorie,      -- Dénormalisé pour éviter la jointure produit
    cl.ville,         -- Dénormalisé pour éviter la jointure client
    (lc.quantite * lc.prix_reel) as montant_vente
FROM lignes_commande lc
JOIN commandes c ON lc.commande_id = c.commande_id
JOIN produits p ON lc.produit_id = p.produit_id
JOIN clients cl ON c.client_id = cl.client_id;
''')

print("✅ Table fact_ventes créée (Simulation DWH).")

# Requête Analytique sur DWH : Beaucoup plus simple !
sql_olap = '''
SELECT 
    strftime('%Y-%m', date_commande) as mois,
    categorie,
    ville,
    SUM(montant_vente) as chiffre_affaires
FROM fact_ventes
GROUP BY 
    strftime('%Y-%m', date_commande), 
    categorie, 
    ville
ORDER BY mois, chiffre_affaires DESC;
'''

print("\n📊 Exécution de la requête OLAP (Simplifiée)...")
df_olap = pd.read_sql_query(sql_olap, conn)
display(df_olap)

## Conclusion et Plan de Transition

Nous avons vu que la modélisation a un impact majeur sur la simplicité des requêtes.

### Ce qu'il faut retenir :
| Critère | OLTP (Transactionnel) | OLAP (Décisionnel) |
|---------|-----------------------|--------------------|
| **Priorité** | Intégrité et Rapidité d'écriture | Rapidité de lecture et Agrégation |
| **Modèle** | Normalisé (Complexe) | Dénormalisé (Étoile/Flocon) |
| **Jointures** | Nombreuses | Minimales |

### Schéma Cible (TD Suivant)
Dans le TD1, nous mettrons en place un véritable **Schéma en Étoile** avec :
- Une table de faits centrale (`fact_ventes`)
- Des tables de dimensions périphériques (`dim_temps`, `dim_produit`, `dim_geo`)

```mermaid
graph LR
    subgraph "Source OLTP"
    S1[Commandes] --> ETL
    S2[Produits] --> ETL
    S3[Clients] --> ETL
    end

    subgraph "Cible DWH (Étoile)"
    ETL --> D1[Dim Temps]
    ETL --> D2[Dim Produit]
    ETL --> D3[Dim Geo]
    ETL --> F1[Fact Ventes]
    
    D1 --- F1
    D2 --- F1
    D3 --- F1
    end
    
    style F1 fill:#f9f,stroke:#333,stroke-width:4px
```

### Prochaines étapes
1. **Audit** : Analyser les sources de données.
2. **Modélisation** : Concevoir le modèle en étoile (TD1).
3. **ETL** : Développer les scripts de chargement.